# Nettoyage

Import

In [2]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

### Connexion Base de donnée Orignale

In [3]:
# Connexion (fichier ou mémoire)
con = duckdb.connect("football.duckdb")
# con = duckdb.connect()  # en mémoire

# Requête SQL
tables = con.execute("DESCRIBE tracking;").fetchall()
print(tables)

df = con.execute("""
    SELECT *
    FROM tracking
    WHERE (player_id, time_utc) IN (
        SELECT player_id, MIN(time_utc)
        FROM tracking
        GROUP BY player_id
    )
""").df()

print(df.head())

[('session_type', 'VARCHAR', 'YES', None, None, None), ('session_date', 'DATE', 'YES', None, None, None), ('player_id', 'INTEGER', 'YES', None, None, None), ('time_utc', 'TIMESTAMP', 'YES', None, None, None), ('x_pos', 'DOUBLE', 'YES', None, None, None), ('y_pos', 'DOUBLE', 'YES', None, None, None), ('latitude', 'DOUBLE', 'YES', None, None, None), ('longitude', 'DOUBLE', 'YES', None, None, None), ('speed_kmh', 'DOUBLE', 'YES', None, None, None), ('heart_rate', 'DOUBLE', 'YES', None, None, None), ('num_sat_fix', 'INTEGER', 'YES', None, None, None), ('hdop', 'DOUBLE', 'YES', None, None, None)]
  session_type session_date  player_id            time_utc      x_pos  \
0     practice   2019-07-08          2 2019-07-08 08:45:22   6.774947   
1     practice   2020-10-21         25 2020-10-21 13:01:19 -16.827745   
2     practice   2019-07-04          6 2019-07-04 08:52:15 -13.249693   
3     practice   2019-07-04         12 2019-07-04 08:52:15 -16.214164   
4     practice   2019-07-04         

### Création nouvelle Table clean en 2 étapes
#### Etape 1 Renommé player_id en sensor_id

In [31]:
# Création d'une nouvelle table nettoyée
#Etape 1 : Renommer player_id en sensor_id
con.execute("""
CREATE TABLE tracking_clean_step1 AS
SELECT
    session_type,
    session_date,
    player_id AS sensor_id,
    time_utc,
    x_pos,
    y_pos,
    latitude,
    longitude,
    speed_kmh,
    heart_rate,
    num_sat_fix,
    hdop
FROM tracking;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Vérification

In [32]:
tables = con.execute("DESCRIBE tracking_clean_step1;").fetchall()
print(tables)

[('session_type', 'VARCHAR', 'YES', None, None, None), ('session_date', 'DATE', 'YES', None, None, None), ('sensor_id', 'INTEGER', 'YES', None, None, None), ('time_utc', 'TIMESTAMP', 'YES', None, None, None), ('x_pos', 'DOUBLE', 'YES', None, None, None), ('y_pos', 'DOUBLE', 'YES', None, None, None), ('latitude', 'DOUBLE', 'YES', None, None, None), ('longitude', 'DOUBLE', 'YES', None, None, None), ('speed_kmh', 'DOUBLE', 'YES', None, None, None), ('heart_rate', 'DOUBLE', 'YES', None, None, None), ('num_sat_fix', 'INTEGER', 'YES', None, None, None), ('hdop', 'DOUBLE', 'YES', None, None, None)]


#### Etape 2 Rajouter la colonne player_id qui correspondras au last name des joueurs

In [33]:
#Etape 2 : Rajouter la colonne player_id (last_name) vierge
con.execute("""CREATE TABLE tracking_clean_step2 AS
SELECT
    session_type,
    session_date,
    NULL::VARCHAR AS player_id,
    sensor_id,
    time_utc,
    x_pos,
    y_pos,
    latitude,
    longitude,
    speed_kmh

FROM tracking_clean_step1;
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Verification

In [34]:
tables = con.execute("DESCRIBE tracking_clean_step2;").fetchall()
print(tables)

df = con.execute("""
    SELECT *
    FROM tracking_clean_step2
    WHERE (sensor_id, time_utc) IN (
        SELECT sensor_id, MIN(time_utc)
        FROM tracking_clean_step2
        GROUP BY sensor_id
    )
""").df()

print(df.head())

[('session_type', 'VARCHAR', 'YES', None, None, None), ('session_date', 'DATE', 'YES', None, None, None), ('player_id', 'VARCHAR', 'YES', None, None, None), ('sensor_id', 'INTEGER', 'YES', None, None, None), ('time_utc', 'TIMESTAMP', 'YES', None, None, None), ('x_pos', 'DOUBLE', 'YES', None, None, None), ('y_pos', 'DOUBLE', 'YES', None, None, None), ('latitude', 'DOUBLE', 'YES', None, None, None), ('longitude', 'DOUBLE', 'YES', None, None, None), ('speed_kmh', 'DOUBLE', 'YES', None, None, None)]
  session_type session_date player_id  sensor_id            time_utc  \
0     practice   2019-07-04      None         10 2019-07-04 08:52:15   
1     practice   2019-07-04      None          5 2019-07-04 08:52:15   
2     practice   2019-07-04      None         13 2019-07-04 08:52:15   
3     practice   2019-07-04      None          7 2019-07-04 08:52:15   
4     practice   2019-07-04      None         23 2019-07-04 08:52:15   

       x_pos      y_pos   latitude  longitude  speed_kmh  
0  -8.5

### Remplissage colonne player_id
à partir du document summarize. Mise en corrélation session_date, sensor_id et player_id

Importation csv summarize

In [35]:
#Lecture fichier CSV
df_csv = pd.read_csv("data/summary.csv", sep=";")

df_csv["session_date"] = pd.to_datetime(
    df_csv["date"],
    dayfirst=True
).dt.date

df_mapping = df_csv.rename(columns={
    "Sensor": "sensor_id",
    "last_name": "player_id"
})[["session_date", "sensor_id", "player_id"]]


In [36]:
con.register("mapping_df", df_mapping)

con.execute("""
CREATE OR REPLACE TABLE sensor_player_session AS
SELECT * FROM mapping_df;
""")

In [37]:
con.execute("""
UPDATE tracking_clean_step2 t
SET player_id = m.player_id
FROM sensor_player_session m
WHERE t.session_date = m.session_date
AND t.sensor_id = m.sensor_id;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [38]:
con.execute("""
SELECT COUNT(*) 
FROM tracking_clean_step2
WHERE player_id IS NULL;
""").fetchone()

(36851,)

In [43]:
df = con.execute("""
    SELECT *
    FROM tracking_clean_step2
    WHERE (player_id, time_utc) IN (
        SELECT player_id, MIN(time_utc)
        FROM tracking_clean_step2
        GROUP BY player_id
    )
""").df()

print(df.head())
print("le nombre de joueurs uniques:", df['player_id'].nunique())

  session_type session_date player_id  sensor_id            time_utc  \
0     practice   2019-07-04     MALES         21 2019-07-04 08:52:15   
1     practice   2019-07-04  CIRKOVIC          3 2019-07-04 08:52:15   
2     practice   2019-07-04   SCHURPF         17 2019-07-04 08:52:15   
3     practice   2019-07-04  DEMHASAJ          1 2019-07-04 08:52:15   
4     practice   2019-07-04    SCHULZ         15 2019-07-04 08:52:15   

       x_pos      y_pos   latitude  longitude  speed_kmh  
0 -12.139701  -1.182093  47.032209   8.306728    0.40536  
1 -12.489483  22.855430  47.032425   8.306724    0.69300  
2 -14.405043   9.191089  47.032302   8.306698    0.43992  
3 -15.096700  -1.359981  47.032207   8.306689    0.08820  
4  -7.168263   2.786979  47.032245   8.306794    1.11924  
le nombre de joueurs uniques: 42


Suppression ligne practice abérant vitesse supérieur à 50km/h

In [4]:
# Nettoyage final : suppression des lignes inutile

con.execute("""
DELETE FROM tracking_clean_step2
WHERE (player_id, time_utc) IN (
    ('SIDLER', '2020-01-24'),
    ('CARBONELL', '2020-11-18'),
    ('IBRAH', '2020-09-16'),
    ('VOCA', '2019-09-14'),
    ('CIRKOVIC', '2019-07-11'),
    ('MALES', '2019-09-18'),
    ('KNEZEVIC', '2020-11-11'),
    ('ELEKE', '2020-03-04'),
    ('KNEZEVIC', '2021-01-08'),
    ('FRYDEK', '2021-01-11'),
    ('SCHWEGLER', '2021-01-08'),
    ('SORGIC', '2021-01-08'),
    ('MARGIOTTA', '2020-02-06'),
    ('SCHAUB', '2021-01-08'),
    ('RYDER', '2020-02-06'),
    ('ALVES', '2021-01-08'),
    ('GRETHER', '2021-01-08'),
    ('SCHURPF', '2020-02-06'),
    ('ALVES', '2020-11-04'),
    ('CARBONELL', '2020-11-04'),
    ('KNEZEVIC', '2020-11-04'),
    ('CIRKOVIC', '2020-02-04'),
    ('ELEKE', '2019-09-18'),
    ('KAKABADZE', '2020-03-13'),
    ('RYDER', '2020-03-13'),
    ('GRETHER', '2020-03-13'),
    ('BÜRKI', '2020-03-13'),
    ('ALVES', '2020-03-13')
);
""")
df = con.execute("""
    SELECT *
    FROM tracking_clean_step2
    WHERE (player_id, time_utc) IN (
        SELECT player_id, MIN(time_utc)
        FROM tracking_clean_step2
        GROUP BY player_id
    )
""").df()
